<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
מקור: [Spark By Examples - PySpark sparkcontext explained](https://sparkbyexamples.com/pyspark/pyspark-sparkcontext-explained/)

# 🚀 PySpark SparkContext - מדריך מקיף בעברית

**מאת:** Naveen Nelamali
**תאריך:** 9 ביולי, 2025
**זמן קריאה משוער:** 10 דקות

---

## 📋 תוכן עניינים

1. [SparkContext ב-PySpark Shell](#1-sparkcontext-ב-pyspark-shell)
2. [יצירת SparkContext ב-PySpark](#2-יצירת-sparkcontext-ב-pyspark)
3. [עצירת SparkContext](#3-עצירת-sparkcontext)
4. [יצירת SparkContext לפני PySpark 2.0](#4-יצירת-sparkcontext-לפני-pyspark-20)
5. [יצירת PySpark RDD](#5-יצירת-pyspark-rdd)
6. [משתנים נפוצים ב-SparkContext](#6-משתנים-נפוצים-ב-sparkcontext)
7. [מתודות נפוצות ב-SparkContext](#7-מתודות-נפוצות-ב-sparkcontext)
8. [סיכום](#8-סיכום)

## 📖 מבוא

**`pyspark.SparkContext`** הוא נקודת הכניסה לפונקציונליות של PySpark. הוא משמש לתקשורת עם האשכול (cluster) וליצירת RDD, Accumulator ומשתני Broadcast.

במדריך זה תלמדו:
- ✅ כיצד ליצור PySpark SparkContext עם דוגמאות
- ✅ שימוש ב-SparkContext ב-PySpark Shell
- ✅ עצירת SparkContext
- ✅ יצירת SparkContext לפני גרסה 2.0
- ✅ משתנים ומתודות נפוצים

> **⚠️ חשוב:** ניתן ליצור רק **SparkContext אחד** ל-JVM. כדי ליצור אחר חדש, יש לעצור את הקיים באמצעות המתודה `stop()`.

---

## 🏗️ ארכיטקטורת SparkContext

תוכנת ה-Driver של Spark יוצרת ומשתמשת ב-SparkContext כדי להתחבר למנהל האשכול (Cluster Manager) לשליחת עבודות PySpark, ולדעת לאיזה מנהל משאבים (YARN, Mesos, או Standalone) לתקשר.

**SparkContext הוא הלב של אפליקציית PySpark!**

```
┌─────────────────┐
│ Driver Program  │
│                 │
│  SparkContext ──┼──► Cluster Manager ──► Worker Nodes
│                 │                          (Executors + Cache + Tasks)
└─────────────────┘
```

*מקור: spark.apache.org*

---

## 1. SparkContext ב-PySpark Shell

כאשר אתם פותחים את ה-PySpark Shell, המערכת יוצרת ומספקת באופן אוטומטי אובייקט `sc`, שהוא מופע של מחלקת SparkContext.

ניתן להשתמש באובייקט זה ישירות ללא צורך ליצור אותו מחדש.

### דוגמה - שימוש ישיר ב-sc

In [ ]:
# ב-PySpark Shell, sc כבר זמין
# אין צורך ליצור אותו
##sc.appName

### 💡 הסבר

ברוב הכלים, המחברות ו-Azure Databricks, הסביבה עצמה יוצרת אובייקט SparkContext ברירת מחדל שאפשר להשתמש בו, כך שאין צורך ליצור הקשר (context) של PySpark.

---

## 2. יצירת SparkContext ב-PySpark

מאז PySark 2.0, יצירת **SparkSession** יוצרת SparkContext באופן פנימי וחושפת את המשתנה `sparkContext` לשימוש.

### דוגמה - יצירה דרך SparkSession

In [ ]:
# יצירת SparkSession מ-builder
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]") \
                    .appName("SparkByExamples.com") \
                    .getOrCreate()

print(spark.sparkContext)
print("Spark App Name : " + spark.sparkContext.appName)

### 📤 פלט צפוי

```
<SparkContext master=local[1] appName=SparkByExamples.com>
Spark App Name : SparkByExamples.com
```

### 💡 הסבר

כפי שהוסבר במאמר על [SparkSession](https://sparkbyexamples.com/pyspark/pyspark-sparksession/), ניתן ליצור כל מספר של אובייקטי SparkSession, אולם עבור כל האובייקטים הללו יהיה רק SparkContext אחד.

### ⚠️ אזהרה - בעיה נפוצה

בכל זמן נתון יכול להיות רק מופע **אחד** של `SparkContext` פעיל ל-JVM. במקרה שתרצו ליצור אחר נוסף, תחילה עליכם לעצור את ה-SparkContext הקיים באמצעות `stop()` לפני יצירת אחד חדש.

---

## 3. עצירת SparkContext

ניתן לעצור את ה-SparkContext על ידי קריאה למתודה `stop()`. 

### דוגמה - עצירת SparkContext

In [ ]:
# מתודת stop() של SparkContext
spark.sparkContext.stop()

### 📤 פלט צפוי

כאשר PySpark מבצע הצהרה זו, הוא רושם את ההודעה:

```
INFO SparkContext: Successfully stopped SparkContext
```

להודעה זו ניתן לראות ב-console או בקובץ log.

### 🚫 שגיאה - ניסיון ליצור SparkContext נוסף

כאשר תנסו ליצור מספר אובייקטי SparkContext, תקבלו את השגיאה הבאה:

```python
ValueError: Cannot run multiple SparkContexts at once;
```

---

## 4. יצירת SparkContext לפני PySpark 2.0

לפני גרסת PySpark 2.0, היה צורך ליצור SparkContext באופן תכנותי באמצעות הקונסטרוקטור, ולהעביר פרמטרים כמו master ו-appName כפרמטרים חובה לפחות.

### דוגמה 1 - שימוש בקונסטרוקטור ישיר

In [ ]:
# יצירת SparkContext
from pyspark import SparkContext

sc = SparkContext("local", "Spark_Example_App")
print(sc.appName)

הדוגמה לעיל יוצרת הקשר עם master כ-`local` ושם האפליקציה כ-`Spark_Example_App`.

### דוגמה 2 - שימוש ב-getOrCreate()

ניתן גם ליצור אותו באמצעות `SparkContext.getOrCreate()`. שיטה זו מחזירה SparkContext קיים פעיל, או יוצרת חדש עם master ושם אפליקציה מוגדרים.

In [ ]:
# יצירת Spark Context
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.setMaster("local").setAppName("Spark_Example_App")
sc = SparkContext.getOrCreate(conf)
print(sc.appName)

---

## 5. יצירת PySpark RDD

ברגע שיש לכם אובייקט SparkContext, תוכלו ליצור PySpark RDD במספר דרכים. להלן השתמשתי בפונקציה `range()` ליצירת RDD.

### דוגמה - יצירת RDD פשוט

In [ ]:
from pyspark.sql import SparkSession

# אתחל סשן חדש (כשה־master=local[*] עובד בסביבת פיתוח מקומית)
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Example") \
    .getOrCreate()

sc = spark.sparkContext
print("SparkContext active:", sc is not None)

# יצירת RDD
rdd = spark.sparkContext.range(1, 5)
print(rdd.collect())

### 📤 פלט

```python
[1, 2, 3, 4]
```

---

## 6. משתנים נפוצים ב-SparkContext

להלן רשימת המשתנים הנפוצים ביותר שזמינים באמצעות SparkContext:

### 🆔 applicationId

מחזיר מזהה ייחודי (Unique ID) של אפליקציית PySpark.

In [ ]:
# קבלת Application ID
print("Application ID: " + spark.sparkContext.applicationId)

### 📌 version

מחזיר את גרסת אשכול PySpark שבה העבודה שלכם רצה.

In [ ]:
# קבלת גרסת Spark
print("Spark Version: " + spark.sparkContext.version)

### 🌐 uiWebUrl

מספק את ה-URL של ממשק המשתמש (Spark Web UI) שהופעל על ידי SparkContext.

In [ ]:
# קבלת Spark UI URL
print("Spark UI URL: " + spark.sparkContext.uiWebUrl)

---

## 7. מתודות נפוצות ב-SparkContext

להלן רשימת המתודות הנפוצות ביותר שזמינות דרך SparkContext:

### 📊 accumulator(value[, accum_param])

יוצרת [משתנה accumulator של PySpark](https://sparkbyexamples.com/pyspark/pyspark-accumulator-with-example/) עם ערך התחלתי שצוין.

רק תוכנת ה-driver יכולה לגשת למשתני accumulator.

### דוגמה

In [ ]:
# יצירת Accumulator
accum = spark.sparkContext.accumulator(0)

# שימוש ב-Accumulator
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
rdd.foreach(lambda x: accum.add(x))

print("Accumulator value: " + str(accum.value))

---

### 📡 broadcast(value)

יוצרת [משתנה broadcast של PySpark](https://sparkbyexamples.com/pyspark/pyspark-broadcast-variables/) לקריאה בלבד.

המשתנה ישודר לכל האשכול. ניתן לשדר משתנה לאשכול PySpark פעם אחת בלבד.

### דוגמה

In [ ]:
# יצירת Broadcast Variable
broadcast_var = spark.sparkContext.broadcast([1, 2, 3])

# שימוש ב-Broadcast Variable
print("Broadcast value: " + str(broadcast_var.value))

---

### 📝 emptyRDD()

יוצרת [RDD ריק](https://sparkbyexamples.com/pyspark/pyspark-create-an-empty-rdd/).

### דוגמה

In [ ]:
# יצירת Empty RDD
empty_rdd = spark.sparkContext.emptyRDD()
print("Is empty: " + str(empty_rdd.isEmpty()))

---

### 🔄 getOrCreate()

יוצרת או מחזירה SparkContext קיים.

### דוגמה

In [ ]:
# קבלה או יצירה של SparkContext
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("MyApp").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)
print(sc.appName)

---

### 📂 hadoopFile()

מחזירה [RDD](https://sparkbyexamples.com/pyspark/pyspark-rdd-tutorial/) של קובץ Hadoop.

### 📂 newAPIHadoopFile()

יוצרת [RDD](https://sparkbyexamples.com/pyspark/pyspark-rdd-tutorial/) עבור קובץ Hadoop עם API חדש של InputFormat.

### 📋 sequenceFile()

מקבלת [RDD](https://sparkbyexamples.com/pyspark/pyspark-rdd-tutorial/) עבור SequenceFile של Hadoop עם מפתח ערך נתונים.

### ⚙️ setLogLevel()

משנה את רמת ה-log ל-debug, info, warn, fatal, ו-error.

### 📄 textFile()

[קוראת קובץ טקסט](https://sparkbyexamples.com/pyspark/pyspark-read-text-file-rdd-dataframe/) מ-HDFS, מקומי או כל מערכת קבצים נתמכת של Hadoop ומחזירה RDD.

---

### 🔗 union()

מאחדת שני RDDs.

### דוגמה

In [ ]:
# איחוד שני RDDs
rdd1 = spark.sparkContext.parallelize([1, 2, 3])
rdd2 = spark.sparkContext.parallelize([4, 5, 6])
union_rdd = rdd1.union(rdd2)
print(union_rdd.collect())

---

### 📁 wholeTextFiles()

[קוראת קובץ טקסט מתיקייה](https://sparkbyexamples.com/pyspark/pyspark-read-text-file-rdd-dataframe/) מ-HDFS, מקומי או כל מערכת קבצים נתמכת של Hadoop ומחזירה RDD של Tuple2.

האלמנט הראשון של ה-tuple מכיל את שם הקובץ והאלמנט השני מכיל את תוכן קובץ הטקסט.

---

## 8. סיכום 🎯

במאמר PySpark Context זה, למדתם:

✅ **מהו SparkContext** - נקודת הכניסה לפונקציונליות PySpark  
✅ **כיצד ליצור אותו** - דרך SparkSession או באופן ישיר  
✅ **כיצד לעצור אותו** - שימוש במתודה `stop()`  
✅ **שימוש עם האשכול** - תקשורת עם Cluster Manager  

באמצעות SparkContext תוכלו ליצור:
- 📊 **RDD** (Resilient Distributed Datasets)
- 📈 **Accumulators** (משתני צבירה)
- 📡 **Broadcast variables** (משתני שידור)

### 💡 זכרו

SparkContext הוא נקודת הכניסה למנוע הביצוע של PySpark אשר מתקשר עם האשכול!

---

## 📚 מאמרים קשורים

- [How to add Multiple Jars to PySpark](https://sparkbyexamples.com/pyspark/)
- [PySpark Read and Write SQL Server Table](https://sparkbyexamples.com/pyspark/pyspark-read-and-write-sql-server-table/)
- [PySpark Read and Write MySQL Database Table](https://sparkbyexamples.com/pyspark/pyspark-read-and-write-mysql-database-table/)
- [PySpark createOrReplaceTempView() Explained](https://sparkbyexamples.com/pyspark/pyspark-createorreplacetempview/)
- [PySpark cache() Explained](https://sparkbyexamples.com/pyspark/pyspark-cache-explained/)
- [PySpark repartition() – Explained with Examples](https://sparkbyexamples.com/pyspark/pyspark-repartition/)
- [PySpark Create RDD with Examples](https://sparkbyexamples.com/pyspark/different-ways-to-create-dataframe-in-pyspark/)
- [PySpark printSchema() Example](https://sparkbyexamples.com/pyspark/pyspark-printschema/)
- [What's New in PySpark 4.0: Features, Improvements, and Enhancements](https://sparkbyexamples.com/pyspark/whats-new-in-pyspark-4-0/)

---

## 🔗 מקורות והפניות

- **תיעוד רשמי:** [SparkContext.scala - Apache Spark GitHub](https://github.com/apache/spark/blob/master/core/src/main/scala/org/apache/spark/SparkContext.scala)
- **אתר המדריך:** [SparkByExamples.com](https://sparkbyexamples.com/)

---

### 🎓 Happy Learning!

**תגיות:** `SQLCONTEXT` | `PYSPARK` | `BIG DATA`

*מדריך זה תורגם ועובד לעברית מתוך המקור באתר SparkByExamples.com*